In [1]:
import cv2
import matplotlib.pyplot as plt
import os

In [5]:
def convert_and_save(input_dir, output_dir):
    """
    Convert images in the input directory to different color spaces and save them in the output directory.

    This function reads each .jpg and .png image in the input directory, converts it to the YUV, HSV, LAB, and HLS color spaces, 
    and saves the converted images in corresponding subdirectories in the output directory.

    Parameters:
    input_dir (str): The path to the input directory containing the images to convert.
    output_dir (str): The path to the output directory where the converted images will be saved.

    Returns:
    None
    """

    # Define the color spaces to convert to
    color_spaces = ['YUV', 'HSV', 'LAB', 'HLS']
    
    # Define the conversion flags for OpenCV
    conversion_flags = {
        'YUV': cv2.COLOR_BGR2YUV,
        'HSV': cv2.COLOR_BGR2HSV,
        'LAB': cv2.COLOR_BGR2LAB,
        'HLS': cv2.COLOR_BGR2HLS,
    }

    # Loop through all files in the input directory
    for filename in os.listdir(input_dir):
        # Check if the file is a .jpg or .png image
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read the image
            img = cv2.imread(os.path.join(input_dir, filename))

            # Convert the image to each color space and save
            for space in color_spaces:
                # Convert the image
                converted_img = cv2.cvtColor(img, conversion_flags[space])
                
                # Define the output subdirectory
                output_subdir = os.path.join(output_dir, space)

                # Create the subdirectory if it doesn't exist
                if not os.path.exists(output_subdir):
                    os.makedirs(output_subdir)

                # Define the output filename
                output_filename = os.path.join(output_subdir, filename)
                
                # Save the converted image
                cv2.imwrite(output_filename, converted_img)

In [6]:
convert_and_save("/Users/titouanlegourrierec/Desktop/PATCHS/RGB", "/Users/titouanlegourrierec/Desktop/PATCHS/RGB")

In [2]:
import os
from skimage import color
from PIL import Image
import numpy as np

def convert_to_eigen_space(image_path):
    # Ouvrir l'image
    img = Image.open(image_path)
    # Convertir l'image en tableau numpy
    rgb_img = np.array(img)
    # Aplatir l'image en une matrice 2D où chaque ligne est une couleur
    rgb_img_2d = rgb_img.reshape(-1, rgb_img.shape[-1])
    # Calculer la matrice de covariance des couleurs
    cov_matrix = np.cov(rgb_img_2d, rowvar=False)
    # Calculer les valeurs propres et les vecteurs propres de la matrice de covariance
    eigen_values, eigen_vectors = np.linalg.eigh(cov_matrix)
    # Trier les vecteurs propres par valeur propre décroissante
    idx = np.argsort(eigen_values)[::-1]
    eigen_vectors = eigen_vectors[:,idx]
    # Transformer l'image dans l'espace Eigen
    eigen_img_2d = np.dot(rgb_img_2d - np.mean(rgb_img_2d, axis=0), eigen_vectors)
    # Remodeler l'image dans sa forme originale
    eigen_img = eigen_img_2d.reshape(rgb_img.shape)
    # Normalize to 0-255
    eigen_img_normalized = (eigen_img - np.min(eigen_img)) / (np.max(eigen_img) - np.min(eigen_img)) * 255
    # Convert to uint8
    eigen_img_uint8 = eigen_img_normalized.astype(np.uint8)
    # Create PIL image
    pil_image = Image.fromarray(eigen_img_uint8)
    return pil_image

def process_directory(input_directory_path, output_directory_path):
    # Get a list of all image files in the directory
    image_files = [f for f in os.listdir(input_directory_path) if f.endswith('.jpg') or f.endswith('.png')]
    # Apply the function to each image file
    for image_file in image_files:
        transformed_image = convert_to_eigen_space(os.path.join(input_directory_path, image_file))
        # Save the transformed image with a modified filename in the output directory
        base, ext = os.path.splitext(image_file)
        transformed_image.save(os.path.join(output_directory_path, f"{base}{ext}"))

In [7]:
process_directory("/Users/titouanlegourrierec/Desktop/PATCHS/RGB", "/Users/titouanlegourrierec/Desktop/PATCHS/Eigen")